In [1]:
import numpy as np
import pandas as pd

In [2]:
from lightautoml.reader.gpu.cudf_reader import CudfReader
from lightautoml.reader.base import PandasToPandasReader

from lightautoml.transformers.base import SequentialTransformer

from lightautoml.pipelines.utils import get_columns_by_role

from lightautoml.transformers.gpu import numeric_gpu, categorical_gpu, datetime_gpu
from lightautoml.transformers import numeric, categorical, datetime

from lightautoml.tasks import Task
from lightautoml.automl.presets.tabular_presets import TabularAutoML, TabularUtilizedAutoML

import pandas as pd
import time
from sklearn.model_selection import train_test_split

from dask.distributed import Client
from dask_cuda import LocalCUDACluster
import cudf

from lightautoml.dataset.roles import TargetRole

import os
import joblib

In [3]:
key = 'airlines'
adv_roles = True
args_fold = 2

data_info = joblib.load(os.path.join("../../data/old_presets", 'data_info.pkl'))[key]
folds = joblib.load(os.path.join("../../data/old_presets", 'folds', '{0}.pkl'.format(key)))

read_csv_params = {}
if 'read_csv_params' in data_info:
    read_csv_params = {**read_csv_params, **data_info['read_csv_params']}

data = pd.read_csv(os.path.join("../../data/old_presets/data", data_info['path']), **read_csv_params)

if 'drop' in data_info:
    data.drop(data_info['drop'], axis=1, inplace=True)

if 'class_map' in data_info:
    data[data_info['target']] = data[data_info['target']].map(data_info['class_map']).values
    assert data[data_info['target']].notnull().all(), 'Class mapping is set unproperly'

print(data.head())
print("task type:", data_info['task_type'])

roles = {TargetRole(): data_info['target']}

  Airline  Flight AirportFrom AirportTo  DayOfWeek  Time  Length  Delay
0      CO     269         SFO       IAH          3    15     205      1
1      US    1558         PHX       CLT          3    15     222      1
2      AA    2400         LAX       DFW          3    20     165      1
3      AA    2466         SFO       DFW          3    20     195      1
4      AS     108         ANC       SEA          3    30     202      0
task type: binary


## Imports (for potential use)

In [4]:
# Imports from our package
from lightautoml.automl.base import AutoML

from lightautoml.automl.presets.gpu.tabular_gpu_presets import TabularAutoMLGPU, TabularUtilizedAutoMLGPU
from lightautoml.tasks import Task

from lightautoml.pipelines.features.gpu.lgb_pipeline_gpu import LGBSimpleFeaturesGPU, LGBAdvancedPipelineGPU
from lightautoml.pipelines.features.gpu.linear_pipeline_gpu import LinearFeaturesGPU

from lightautoml.pipelines.features.lgb_pipeline import LGBSimpleFeatures, LGBAdvancedPipeline
from lightautoml.pipelines.features.linear_pipeline import LinearFeatures


from lightautoml.ml_algo.gpu.boost_cb_gpu import BoostCBGPU
from lightautoml.ml_algo.gpu.boost_xgb_gpu import BoostXGB
from lightautoml.ml_algo.gpu.linear_gpu import LinearLBFGSGPU

from lightautoml.ml_algo.boost_cb import BoostCB
from lightautoml.ml_algo.linear_sklearn import LinearLBFGS


from lightautoml.pipelines.ml.base import MLPipeline
from lightautoml.pipelines.selection.importance_based import ModelBasedImportanceEstimator, ImportanceCutoffSelector

## TabularAutoML

In [5]:
task = Task(data_info['task_type'])

In [6]:
automl = TabularAutoML(
    task = task, 
    timeout = 3600,
    cpu_limit = 4,
    reader_params = {'n_jobs': 4, 'cv': 3, 'random_state': 42},
    general_params = {'use_algos': ['linear_l2', 'cb', 'lgbm']}
)

In [7]:
cpu_fit_pred = automl.fit_predict(data[folds!=args_fold].reset_index().drop(['index'],axis=1), roles = roles, verbose = 2)

[14:20:56] Stdout logging level is INFO2.
[14:20:56] Copying TaskTimer may affect the parent PipelineTimer, so copy will create new unlimited TaskTimer
[14:20:56] Task: binary

[14:20:56] Start automl preset with listed constraints:
[14:20:56] - time: 3600.00 seconds
[14:20:56] - CPU: 4 cores
[14:20:56] - memory: 16 GB

[14:20:56] Train data shape: (431506, 8)

[14:21:00] Feats was rejected during automatic roles guess: []
[14:21:00] Layer 1 train process start. Time left 3595.81 secs
[14:21:11] Start fitting Lvl_0_Pipe_0_Mod_0_LinearL2 ...
[14:21:11] ===== Start working with fold 0 for Lvl_0_Pipe_0_Mod_0_LinearL2 =====
[14:21:14] ===== Start working with fold 1 for Lvl_0_Pipe_0_Mod_0_LinearL2 =====
[14:21:16] ===== Start working with fold 2 for Lvl_0_Pipe_0_Mod_0_LinearL2 =====
[14:21:19] Fitting Lvl_0_Pipe_0_Mod_0_LinearL2 finished. score = 0.7079384207630041
[14:21:19] Lvl_0_Pipe_0_Mod_0_LinearL2 fitting and predicting completed
[14:21:19] Time left 3576.78 secs

[14:21:19] Layer 1 

/home/rishat/miniconda3/envs/rapids-22.10/lib/python3.9/site-packages/catboost/core.py:1419: FutureWarning: iteritems is deprecated and will be removed in a future version. Use .items instead.
  self._init_pool(data, label, cat_features, text_features, embedding_features, embedding_features_data, pairs, weight,
/home/rishat/miniconda3/envs/rapids-22.10/lib/python3.9/site-packages/catboost/core.py:1419: FutureWarning: iteritems is deprecated and will be removed in a future version. Use .items instead.
  self._init_pool(data, label, cat_features, text_features, embedding_features, embedding_features_data, pairs, weight,
Default metric period is 5 because AUC is/are not implemented for GPU


[14:21:46] ===== Start working with fold 1 for Lvl_1_Pipe_0_Mod_0_CatBoost =====


/home/rishat/miniconda3/envs/rapids-22.10/lib/python3.9/site-packages/catboost/core.py:1419: FutureWarning: iteritems is deprecated and will be removed in a future version. Use .items instead.
  self._init_pool(data, label, cat_features, text_features, embedding_features, embedding_features_data, pairs, weight,
/home/rishat/miniconda3/envs/rapids-22.10/lib/python3.9/site-packages/catboost/core.py:1419: FutureWarning: iteritems is deprecated and will be removed in a future version. Use .items instead.
  self._init_pool(data, label, cat_features, text_features, embedding_features, embedding_features_data, pairs, weight,
Default metric period is 5 because AUC is/are not implemented for GPU


[14:21:54] ===== Start working with fold 2 for Lvl_1_Pipe_0_Mod_0_CatBoost =====


/home/rishat/miniconda3/envs/rapids-22.10/lib/python3.9/site-packages/catboost/core.py:1419: FutureWarning: iteritems is deprecated and will be removed in a future version. Use .items instead.
  self._init_pool(data, label, cat_features, text_features, embedding_features, embedding_features_data, pairs, weight,
/home/rishat/miniconda3/envs/rapids-22.10/lib/python3.9/site-packages/catboost/core.py:1419: FutureWarning: iteritems is deprecated and will be removed in a future version. Use .items instead.
  self._init_pool(data, label, cat_features, text_features, embedding_features, embedding_features_data, pairs, weight,
Default metric period is 5 because AUC is/are not implemented for GPU


[14:22:01] Fitting Lvl_1_Pipe_0_Mod_0_CatBoost finished. score = 0.7167728338119556
[14:22:01] Lvl_1_Pipe_0_Mod_0_CatBoost fitting and predicting completed
[14:22:01] Time left 3535.22 secs

[14:22:01] Layer 2 training completed.

[14:22:01] Layer 3 train process start. Time left 3535.20 secs
[14:22:13] Start fitting Lvl_2_Pipe_0_Mod_0_LightGBM ...
[14:22:13] ===== Start working with fold 0 for Lvl_2_Pipe_0_Mod_0_LightGBM =====
[14:22:18] ===== Start working with fold 1 for Lvl_2_Pipe_0_Mod_0_LightGBM =====
[14:22:24] ===== Start working with fold 2 for Lvl_2_Pipe_0_Mod_0_LightGBM =====
[14:22:32] Fitting Lvl_2_Pipe_0_Mod_0_LightGBM finished. score = 0.7176078633171086
[14:22:32] Lvl_2_Pipe_0_Mod_0_LightGBM fitting and predicting completed
[14:22:32] Time left 3504.09 secs

[14:22:32] Layer 3 training completed.

[14:22:32] Automl preset training completed in 95.92 seconds

[14:22:32] Model description:
Models on level 0:
	 3 averaged models Lvl_0_Pipe_0_Mod_0_LinearL2

Models on level

In [8]:
cpu_pred = automl.predict(data[folds==args_fold].reset_index().drop(['index'],axis=1))

/home/rishat/miniconda3/envs/rapids-22.10/lib/python3.9/site-packages/catboost/core.py:1419: FutureWarning: iteritems is deprecated and will be removed in a future version. Use .items instead.
  self._init_pool(data, label, cat_features, text_features, embedding_features, embedding_features_data, pairs, weight,
/home/rishat/miniconda3/envs/rapids-22.10/lib/python3.9/site-packages/catboost/core.py:1419: FutureWarning: iteritems is deprecated and will be removed in a future version. Use .items instead.
  self._init_pool(data, label, cat_features, text_features, embedding_features, embedding_features_data, pairs, weight,
/home/rishat/miniconda3/envs/rapids-22.10/lib/python3.9/site-packages/catboost/core.py:1419: FutureWarning: iteritems is deprecated and will be removed in a future version. Use .items instead.
  self._init_pool(data, label, cat_features, text_features, embedding_features, embedding_features_data, pairs, weight,


In [9]:
task = Task(data_info['task_type'], device='gpu')

In [10]:
automl_gpu = TabularAutoMLGPU(
    task = task, 
    timeout = 3600,
    cpu_limit = 1,
    reader_params = {'n_jobs': 1, 'cv': 3, 'random_state': 42},
    general_params = {'use_algos': ['linear_l2', 'xgb', 'cb']}
)

In [11]:
gpu_fit_pred = automl_gpu.fit_predict(data[folds!=args_fold].reset_index().drop(['index'],axis=1), roles = roles, verbose = 2)

[14:22:39] Stdout logging level is INFO2.
[14:22:39] Task: binary

[14:22:39] Start automl preset with listed constraints:
[14:22:39] - time: 3600.00 seconds
[14:22:39] - CPU: 1 cores
[14:22:39] - memory: 16 GB

[14:22:39] Train data shape: (431506, 8)
Feats was rejected during automatic roles guess: []
[14:22:40] Layer 1 train process start. Time left 3599.34 secs
[14:22:41] Start fitting Lvl_0_Pipe_0_Mod_0_LinearL2 ...
[14:22:41] ===== Start working with fold 0 for Lvl_0_Pipe_0_Mod_0_LinearL2 (orig) =====
[14:22:47] ===== Start working with fold 1 for Lvl_0_Pipe_0_Mod_0_LinearL2 (orig) =====
[14:22:50] ===== Start working with fold 2 for Lvl_0_Pipe_0_Mod_0_LinearL2 (orig) =====
[14:22:54] Fitting Lvl_0_Pipe_0_Mod_0_LinearL2 finished. score = 0.7083783149719238
[14:22:54] Lvl_0_Pipe_0_Mod_0_LinearL2 fitting and predicting completed
[14:22:54] Time left 3584.54 secs

[14:22:54] Layer 1 training completed.

[14:22:54] Layer 2 train process start. Time left 3584.54 secs


/home/rishat/miniconda3/envs/rapids-22.10/lib/python3.9/site-packages/catboost/core.py:1419: FutureWarning: iteritems is deprecated and will be removed in a future version. Use .items instead.
  self._init_pool(data, label, cat_features, text_features, embedding_features, embedding_features_data, pairs, weight,
/home/rishat/miniconda3/envs/rapids-22.10/lib/python3.9/site-packages/catboost/core.py:1419: FutureWarning: iteritems is deprecated and will be removed in a future version. Use .items instead.
  self._init_pool(data, label, cat_features, text_features, embedding_features, embedding_features_data, pairs, weight,
Default metric period is 5 because AUC is/are not implemented for GPU


[14:23:06] Selector_CatBoostGPU fitting and predicting completed
[14:23:07] Start fitting Lvl_1_Pipe_0_Mod_0_XGB ...
[14:23:07] ===== Start working with fold 0 for Lvl_1_Pipe_0_Mod_0_XGB (orig) =====
[14:23:14] ===== Start working with fold 1 for Lvl_1_Pipe_0_Mod_0_XGB (orig) =====
[14:23:22] ===== Start working with fold 2 for Lvl_1_Pipe_0_Mod_0_XGB (orig) =====
[14:23:33] Fitting Lvl_1_Pipe_0_Mod_0_XGB finished. score = 0.7159843444824219
[14:23:33] Lvl_1_Pipe_0_Mod_0_XGB fitting and predicting completed
[14:23:33] Time left 3546.40 secs

[14:23:33] Layer 2 training completed.

[14:23:33] Layer 3 train process start. Time left 3546.40 secs
[14:23:34] Start fitting Lvl_2_Pipe_0_Mod_0_CatBoostGPU ...
[14:23:34] ===== Start working with fold 0 for Lvl_2_Pipe_0_Mod_0_CatBoostGPU (orig) =====


/home/rishat/miniconda3/envs/rapids-22.10/lib/python3.9/site-packages/catboost/core.py:1419: FutureWarning: iteritems is deprecated and will be removed in a future version. Use .items instead.
  self._init_pool(data, label, cat_features, text_features, embedding_features, embedding_features_data, pairs, weight,
/home/rishat/miniconda3/envs/rapids-22.10/lib/python3.9/site-packages/catboost/core.py:1419: FutureWarning: iteritems is deprecated and will be removed in a future version. Use .items instead.
  self._init_pool(data, label, cat_features, text_features, embedding_features, embedding_features_data, pairs, weight,
Default metric period is 5 because AUC is/are not implemented for GPU


[14:23:39] ===== Start working with fold 1 for Lvl_2_Pipe_0_Mod_0_CatBoostGPU (orig) =====


/home/rishat/miniconda3/envs/rapids-22.10/lib/python3.9/site-packages/catboost/core.py:1419: FutureWarning: iteritems is deprecated and will be removed in a future version. Use .items instead.
  self._init_pool(data, label, cat_features, text_features, embedding_features, embedding_features_data, pairs, weight,
/home/rishat/miniconda3/envs/rapids-22.10/lib/python3.9/site-packages/catboost/core.py:1419: FutureWarning: iteritems is deprecated and will be removed in a future version. Use .items instead.
  self._init_pool(data, label, cat_features, text_features, embedding_features, embedding_features_data, pairs, weight,
Default metric period is 5 because AUC is/are not implemented for GPU


[14:23:48] ===== Start working with fold 2 for Lvl_2_Pipe_0_Mod_0_CatBoostGPU (orig) =====


/home/rishat/miniconda3/envs/rapids-22.10/lib/python3.9/site-packages/catboost/core.py:1419: FutureWarning: iteritems is deprecated and will be removed in a future version. Use .items instead.
  self._init_pool(data, label, cat_features, text_features, embedding_features, embedding_features_data, pairs, weight,
/home/rishat/miniconda3/envs/rapids-22.10/lib/python3.9/site-packages/catboost/core.py:1419: FutureWarning: iteritems is deprecated and will be removed in a future version. Use .items instead.
  self._init_pool(data, label, cat_features, text_features, embedding_features, embedding_features_data, pairs, weight,
Default metric period is 5 because AUC is/are not implemented for GPU


[14:23:56] Fitting Lvl_2_Pipe_0_Mod_0_CatBoostGPU finished. score = 0.7170827984809875
[14:23:56] Lvl_2_Pipe_0_Mod_0_CatBoostGPU fitting and predicting completed
[14:23:56] Time left 3523.13 secs

[14:23:56] Layer 3 training completed.

[14:23:56] Automl preset training completed in 76.87 seconds

[14:23:56] Model description:
Models on level 0:
	 3 averaged models Lvl_0_Pipe_0_Mod_0_LinearL2

Models on level 1:
	 3 averaged models Lvl_1_Pipe_0_Mod_0_XGB

Final prediction for new objects (level 2) = 
	 1.00000 * (3 averaged models Lvl_2_Pipe_0_Mod_0_CatBoostGPU) 



In [12]:
gpu_inf = automl_gpu.predict(data[folds==args_fold].reset_index().drop(['index'],axis=1))

/home/rishat/miniconda3/envs/rapids-22.10/lib/python3.9/site-packages/catboost/core.py:1419: FutureWarning: iteritems is deprecated and will be removed in a future version. Use .items instead.
  self._init_pool(data, label, cat_features, text_features, embedding_features, embedding_features_data, pairs, weight,
/home/rishat/miniconda3/envs/rapids-22.10/lib/python3.9/site-packages/catboost/core.py:1419: FutureWarning: iteritems is deprecated and will be removed in a future version. Use .items instead.
  self._init_pool(data, label, cat_features, text_features, embedding_features, embedding_features_data, pairs, weight,
/home/rishat/miniconda3/envs/rapids-22.10/lib/python3.9/site-packages/catboost/core.py:1419: FutureWarning: iteritems is deprecated and will be removed in a future version. Use .items instead.
  self._init_pool(data, label, cat_features, text_features, embedding_features, embedding_features_data, pairs, weight,


In [13]:
automl_gpu.to_cpu()

Models parameters: {'_init_params': {'task_type': 'GPU', 'devices': '0', 'thread_count': 1, 'random_seed': 42, 'learning_rate': 0.05, 'l2_leaf_reg': 0.01, 'bootstrap_type': 'Bernoulli', 'grow_policy': 'SymmetricTree', 'max_depth': 5, 'min_data_in_leaf': 1, 'one_hot_max_size': 10, 'fold_permutation_block': 1, 'boosting_type': 'Plain', 'od_type': 'Iter', 'max_bin': 32, 'feature_border_type': 'GreedyLogSum', 'nan_mode': 'Min', 'verbose': False, 'max_ctr_complexity': 1, 'num_trees': 3000, 'objective': 'Logloss', 'eval_metric': 'AUC', 'od_wait': 100}, '_object': <_catboost._CatBoost object at 0x7fcd68f53400>, '_is_fitted_': True, '_random_seed': 42, '_learning_rate': 0.05000000074505806, '_tree_count': 774, '_n_features_in': 20, '_prediction_values_change': [2.1750902691001297, 5.076168164338567, 3.220156124006821, 2.9674498896177623, 2.5282828208675676, 3.2260336098832862, 5.2225764808514645, 1.1118635684449916, 1.7029795599176003, 2.911652531558489, 3.6668095603794573, 5.877746382719845, 

In [14]:
cpu_inf = automl_gpu.predict(data[folds==args_fold].reset_index().drop(['index'],axis=1))

/home/rishat/miniconda3/envs/rapids-22.10/lib/python3.9/site-packages/catboost/core.py:1419: FutureWarning: iteritems is deprecated and will be removed in a future version. Use .items instead.
  self._init_pool(data, label, cat_features, text_features, embedding_features, embedding_features_data, pairs, weight,
/home/rishat/miniconda3/envs/rapids-22.10/lib/python3.9/site-packages/catboost/core.py:1419: FutureWarning: iteritems is deprecated and will be removed in a future version. Use .items instead.
  self._init_pool(data, label, cat_features, text_features, embedding_features, embedding_features_data, pairs, weight,
/home/rishat/miniconda3/envs/rapids-22.10/lib/python3.9/site-packages/catboost/core.py:1419: FutureWarning: iteritems is deprecated and will be removed in a future version. Use .items instead.
  self._init_pool(data, label, cat_features, text_features, embedding_features, embedding_features_data, pairs, weight,


In [15]:
cluster = LocalCUDACluster(rmm_managed_memory=True, CUDA_VISIBLE_DEVICES="0",
                               protocol="ucx", enable_nvlink=True,
                               memory_limit="8GB")
print("dashboard:", cluster.dashboard_link)
client = Client(cluster)
client.run(cudf.set_allocator, "managed")

/home/rishat/miniconda3/envs/rapids-22.10/lib/python3.9/site-packages/distributed/comm/ucx.py:61: UserWarning: A CUDA context for device 0 already exists on process ID 21226. This is often the result of a CUDA-enabled library calling a CUDA runtime function before Dask-CUDA can spawn worker processes. Please make sure any such function calls don't happen at import time or in the global scope of a program.
  warnings.warn(
2022-12-07 14:24:01,301 - distributed.diskutils - INFO - Found stale lock file and directory '/tmp/dask-worker-space/worker-tzrfwv9q', purging
2022-12-07 14:24:01,306 - distributed.preloading - INFO - Creating preload: dask_cuda.initialize
2022-12-07 14:24:01,306 - distributed.preloading - INFO - Import preload module: dask_cuda.initialize


dashboard: http://127.0.0.1:8787/status


{'ucx://127.0.0.1:41589': None}

In [16]:
task = Task(data_info['task_type'], device='mgpu')

automl_mgpu = TabularAutoMLGPU(
    task = task, 
    timeout = 3600,
    cpu_limit = 1,
    reader_params = {'n_jobs': 1, 'cv': 3, 'random_state': 42, 'npartitions': 2},
    general_params = {'use_algos': ['linear_l2', 'xgb', 'cb']},
    client = client
)

mgpu_fit_pred = automl_mgpu.fit_predict(data[folds!=args_fold].reset_index().drop(['index'],axis=1), roles = roles, verbose = 2)

[14:24:01] Stdout logging level is INFO2.
[14:24:01] Task: binary

[14:24:01] Start automl preset with listed constraints:
[14:24:01] - time: 3600.00 seconds
[14:24:01] - CPU: 1 cores
[14:24:01] - memory: 16 GB

[14:24:01] Train data shape: (431506, 8)
Feats was rejected during automatic roles guess: []
[14:24:02] Layer 1 train process start. Time left 3599.41 secs
[14:24:03] Start fitting Lvl_0_Pipe_0_Mod_0_LinearL2 ...
[14:24:03] ===== Start working with fold 0 for Lvl_0_Pipe_0_Mod_0_LinearL2 (orig) =====
[14:24:07] ===== Start working with fold 1 for Lvl_0_Pipe_0_Mod_0_LinearL2 (orig) =====
[14:24:11] ===== Start working with fold 2 for Lvl_0_Pipe_0_Mod_0_LinearL2 (orig) =====
[14:24:14] Fitting Lvl_0_Pipe_0_Mod_0_LinearL2 finished. score = 0.7083818316459656
[14:24:14] Lvl_0_Pipe_0_Mod_0_LinearL2 fitting and predicting completed
[14:24:14] Time left 3587.18 secs

[14:24:14] Layer 1 training completed.

[14:24:14] Layer 2 train process start. Time left 3587.18 secs


/home/rishat/miniconda3/envs/rapids-22.10/lib/python3.9/site-packages/catboost/core.py:1419: FutureWarning: iteritems is deprecated and will be removed in a future version. Use .items instead.
  self._init_pool(data, label, cat_features, text_features, embedding_features, embedding_features_data, pairs, weight,
/home/rishat/miniconda3/envs/rapids-22.10/lib/python3.9/site-packages/catboost/core.py:1419: FutureWarning: iteritems is deprecated and will be removed in a future version. Use .items instead.
  self._init_pool(data, label, cat_features, text_features, embedding_features, embedding_features_data, pairs, weight,
Default metric period is 5 because AUC is/are not implemented for GPU


[14:24:26] Selector_CatBoostGPU fitting and predicting completed
[14:24:27] Start fitting Lvl_1_Pipe_0_Mod_0_XGB ...
[14:24:27] ===== Start working with fold 0 for Lvl_1_Pipe_0_Mod_0_XGB (orig) =====
THIS IS MGPU


/home/rishat/miniconda3/envs/rapids-22.10/lib/python3.9/site-packages/xgboost/dask.py:884: RuntimeWarning: coroutine 'Client._wait_for_workers' was never awaited
  client.wait_for_workers(n_workers)
/home/rishat/miniconda3/envs/rapids-22.10/lib/python3.9/site-packages/distributed/worker_state_machine.py:3649: FutureWarning: The `Worker.nthreads` attribute has been moved to `Worker.state.nthreads`
  warnings.warn(
[14:24:31] task [xgboost.dask-0]:ucx://127.0.0.1:41589 got new rank 0


[14:24:32] Model Lvl_1_Pipe_0_Mod_0_XGB failed during ml_algo.fit_predict call.

uninitialized_fill_n: failed to synchronize: cudaErrorIllegalAddress: an illegal memory access was encountered


2022-12-07 14:24:32,045 - distributed.worker - WARNING - Compute Failed
Key:       dispatched_train-2b58ca9a-4715-4aca-a900-fe05436406e5
Function:  dispatched_train
args:      ({'tree_method': 'gpu_hist', 'predictor': 'gpu_predictor', 'task': 'train', 'learning_rate': 0.05, 'max_leaves': 244, 'max_depth': 0, 'verbosity': 0, 'reg_alpha': 1, 'reg_lambda': 0.0, 'gamma': 0.0, 'max_bin': 255, 'random_state': 42, 'nthread': 1, 'objective': 'binary:logistic', 'metric': 'auc', 'num_class': 1}, [b'DMLC_NUM_WORKER=1', b'DMLC_TRACKER_URI=192.168.18.5', b'DMLC_TRACKER_PORT=60433', b'DMLC_TASK_ID=[xgboost.dask-0]:ucx://127.0.0.1:41589'], 140519969677216, ['train', 'valid'], [140519969677216, 140518548399920], < could not convert arg to str >, < could not convert arg to str >, < could not convert arg to str >)
kwargs:    {}
Exception: "XGBoostError('uninitialized_fill_n: failed to synchronize: cudaErrorIllegalAddress: an illegal memory access was encountered')"



AssertionError: Pipeline finished with 0 models for some reason.
Probably one or more models failed

In [ ]:
mgpu_pred = automl_mgpu.predict(data[folds==args_fold].reset_index().drop(['index'],axis=1))

In [ ]:
print(cpu_inf.data.T)
print()
print(gpu_inf.data.T)
print()
print(cpu_pred.data.T)
print()
print(mgpu_pred.data.T)

In [ ]:
print(cpu_fit_pred.data.T)
print()
print(gpu_fit_pred.data.T)
print()
print(mgpu_fit_pred.data.T)

In [ ]:
automl_mgpu.to_cpu()

In [ ]:
mgpu_inf = automl_mgpu.predict(data[folds==args_fold].reset_index().drop(['index'],axis=1))

In [ ]:
print(mgpu_inf.data.T)

terminate called after throwing an instance of 'thrust::system::system_error'
  what():  parallel_for failed: cudaErrorIllegalAddress: an illegal memory access was encountered
2022-12-07 14:24:56,856 - distributed.nanny - WARNING - Restarting worker
2022-12-07 14:24:57,983 - distributed.preloading - INFO - Creating preload: dask_cuda.initialize
2022-12-07 14:24:57,983 - distributed.preloading - INFO - Import preload module: dask_cuda.initialize
